# Compute ICC

This notebook computes the intra-class correlation.

References:<br />

Gelman, A., & Hill, J. (2006). Data Analysis Using Regression and Multilevel/Hierarchical Models (1st ed.). Cambridge University Press.

#### Import statements

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas
import numpy
from os import path

In [3]:
%matplotlib inline
from matplotlib import pyplot
from matplotlib.patches import Rectangle
import seaborn

In [4]:
from pymc3 import glm, Model, NUTS, sample, stats, \
                  forestplot, traceplot, plot_posterior, summary, \
                  Normal, Uniform, Deterministic, StudentT
from pymc3.backends import Text

In [5]:
from figures.figutils import style

In [6]:
style.set_font()

In [7]:
seaborn.set()

#### Plot settings

#### Variable definitions

In [8]:
data_filepath = r"C:\Users\JDDein\OneDrive\Documents\alpine soundscapes\data\dataset_r.csv"

In [9]:
trace_output_path = r"C:\Users\JDDein\OneDrive\Documents\alpine soundscapes\data\model traces\sel"

In [10]:
seaborn_blue = seaborn.color_palette()[0]

## Load data

In [11]:
data = pandas.read_csv(data_filepath)
data = data.loc[data.site<=30]

sort data by site and then by visit

In [12]:
data_sorted = data.sort_values(by=['site', 'sound']).reset_index(drop=True)

create sites variable for PyMC3 models

In [13]:
sites = numpy.copy(data_sorted.site.values) - 1

## Model 0 - emtpy model

$$
\begin{align}
y_{ts} \sim \mathcal{N}(\alpha_s + \epsilon_t, \sigma_y^2) \\
\alpha_s \sim \mathcal{N}(M + \epsilon_s, \sigma_\alpha^2) \\
\end{align}
$$

In [14]:
with Model() as model0:
    
    # Priors
    mu_grand = Normal('mu_grand', mu=0., tau=0.0001)
    sigma_a = Uniform('sigma_a', lower=0, upper=100)
    tau_a = sigma_a**-2
    
    # Random intercepts
    a = Normal('a', mu=mu_grand, tau=tau_a, shape=len(set(sites)))
    
    # Model error
    sigma_y = Uniform('sigma_y', lower=0, upper=100)
    tau_y = sigma_y**-2
    
    # Expected value
    y_hat = a[sites]
    
    # Data likelihood
    y_like = Normal('y_like', mu=y_hat, tau=tau_y, observed=data_sorted.sel)
    
    # sample model
    backend = Text(path.join(trace_output_path, "model0"))
    model0_samples = sample(draws=5000, random_seed=1, trace=backend, 
                            discard_tuned_samples=True, target_accept=0.95, tune=1000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma_y, a, sigma_a, mu_grand]
Sampling 4 chains: 100%|█████████████| 24000/24000 [00:48<00:00, 491.61draws/s]
